# Pre-processing:
## Generating config files

In [1]:
import numpy as np 
import pandas as pd
import os

In [5]:
#connect to the server and copy the path to your sequencing data
fp = '/Volumes/sanchezrivera/samgould/240624San/'
k = os.listdir(fp)

#might need to change the 'D24' depending on file naming
k = [i for i in k if i[:3]=='D24'] 

folder_name = []
f1 = []
f2 = []
for ii in range(len(k)):
    j = os.listdir(f'{fp}/{k[ii]}')
    j = sorted([i for i in j if i[-5:]=='fastq'])

    #weird error to deal with
    #hacky solution
    if len(j)==2:
        f1.append(f'./{k[ii]}/{j[0]}')
        f2.append(f'./{k[ii]}/{j[1]}')
    else:
        f1.append(f'./{k[ii]}/{j[2]}')
        f2.append(f'./{k[ii]}/{j[3]}')
    folder_name.append(k[ii] + '_guide_split_BARCODES')

config_df = pd.DataFrame(dict(zip(['ArrayTaskID', 'R1_FILE', 'R2_FILE', 'folder_name'], [list(range(1, len(f1)+1)), f1, f2, folder_name])))
#config_df.to_csv('sensor_extraction_fastq_split_config_DIEGO_IDR.txt', index=False, sep = ' ')

In [8]:
config_df

,ArrayTaskID,R1_FILE,R2_FILE,folder_name
0,1,./D24-10888-6775T/240624San_D24-10888_1_sequen...,./D24-10888-6775T/240624San_D24-10888_2_sequen...,D24-10888-6775T_guide_split_BARCODES
1,2,./D24-10889-6775T/240624San_D24-10889_1_sequen...,./D24-10889-6775T/240624San_D24-10889_2_sequen...,D24-10889-6775T_guide_split_BARCODES
2,3,./D24-10890-6775T/240624San_D24-10890_1_sequen...,./D24-10890-6775T/240624San_D24-10890_2_sequen...,D24-10890-6775T_guide_split_BARCODES
3,4,./D24-10891-6775T/240624San_D24-10891_1_sequen...,./D24-10891-6775T/240624San_D24-10891_2_sequen...,D24-10891-6775T_guide_split_BARCODES
4,5,./D24-10892-6775T/240624San_D24-10892_1_sequen...,./D24-10892-6775T/240624San_D24-10892_2_sequen...,D24-10892-6775T_guide_split_BARCODES
5,6,./D24-10893-6775T/240624San_D24-10893_1_sequen...,./D24-10893-6775T/240624San_D24-10893_2_sequen...,D24-10893-6775T_guide_split_BARCODES


# Example post-processing

In [2]:
def crispresso_compiler(samp, sample_id, fp):
    """ 
    Takes in list of sample names (see cell above)
    Returns compiled dictionary of dataframes containing editing information for each sensor
    - key for each is the sample_id name
    
    Includes:
    1. corr_perc = pure correct editing
    2. target_base_edit_perc = target base editing perc (including edits with bystander editing)
    3. wt_perc
    4. byproduct information (indels, substitutions, ambiguous)
    """

    df_edits = []
    for k in samp: 
        concated = pd.read_csv(f'{fp}/{k}_crispresso_aggregated.csv')
        concated = concated.fillna(0)

        #somehow wasn't able to find the initial set of code that I used for this...

        #go through all of the samples and do it step by step
        sample_ids = []
        #sample_num = []
        rii = []
        raaa = []
        r_lowqual = []
        ra_hdr = []
        ra_wt = []
        r_unaligned = []
        no_edit = []
        correct = []
        target_base_editing = []
        byproduct_all = []
        byproduct_indel = []
        byproduct_sub = []
        byproduct_ambig = []
        for i in np.unique(concated['Guide_ID']):
            sample_ids.append(i)

            subset = concated[concated['Guide_ID']==i]

            wt = subset[subset['Amplicon']=='Reference']
            edit = subset[subset['Amplicon']=='HDR']

            #sample_num.append(wt['sample_num'].values[0])
            rii1 = wt['Reads_in_input'].values[0]
            r1 = wt['Reads_aligned_all_amplicons'].values[0]
            rii.append(rii1)
            raaa.append(r1)
            r_lowqual.append(rii1-r1)

            r2 = edit['Reads_aligned'].values[0]
            r3 = wt['Reads_aligned'].values[0]
            ra_hdr.append(r2)
            ra_wt.append(r3)
            r_unaligned.append(r1 - (r2+r3))

            no_edit.append(wt['Unmodified'].values[0])
            correct.append(edit['Unmodified'].values[0])
            target_base_editing.append(edit['Modified'].values[0] + edit['Unmodified'].values[0])

            byprod_all = wt['Modified'].values[0] + edit['Modified'].values[0] + (r1 - (r2+r3)) #add unaligned reads
            sub_all = wt['Only Substitutions'].values[0] + edit['Only Substitutions'].values[0]
            indel_all = wt['Only Deletions'].values[0] + wt['Only Insertions'].values[0] + wt['Insertions and Deletions'].values[0] + edit['Only Deletions'].values[0] + edit['Only Insertions'].values[0] + edit['Insertions and Deletions'].values[0]
            ambig_all = byprod_all - sub_all - indel_all

            byproduct_all.append(byprod_all)
            byproduct_indel.append(indel_all)
            byproduct_sub.append(sub_all)
            byproduct_ambig.append(ambig_all)


        cols = ["Guide_ID", "Reads_in_input","Reads_lowqual", "Reads_aligned_all_amplicons","Reads_aligned_WT", "Reads_aligned_HDR", "Reads_unaligned", "WT","correct_edit", "target_base_edit", "byproduct_all","byproduct_INDEL","byproduct_sub","byproduct_ambiguous"]
        col_vals = [ sample_ids, rii, r_lowqual, raaa, ra_wt, ra_hdr, r_unaligned, no_edit, correct, target_base_editing, byproduct_all, byproduct_indel, byproduct_sub, byproduct_ambig]

        out = pd.DataFrame(dict(zip(cols, col_vals)))
        out['corr_perc'] = 100*(out['correct_edit']/out['Reads_aligned_all_amplicons'])
        out['target_base_edit_perc'] = 100*(out['target_base_edit']/out['Reads_aligned_all_amplicons'])
        out['WT_perc'] = 100*(out['WT']/out['Reads_aligned_all_amplicons'])
        out['byproduct_all_perc'] =  100*(out['byproduct_all']/out['Reads_aligned_all_amplicons'])
        out['byproduct_INDEL_perc'] =  100*(out['byproduct_INDEL']/out['Reads_aligned_all_amplicons'])
        out['byproduct_sub_perc'] =  100*(out['byproduct_sub']/out['Reads_aligned_all_amplicons'])
        out['byproduct_ambiguous_perc'] =100*(out['byproduct_ambiguous']/out['Reads_aligned_all_amplicons'])
        out = out.fillna(0)

        df_edits.append(out)

    edit_dict = dict(zip(sample_id, df_edits))

    return edit_dict

In [9]:
config = pd.read_csv('example_config_and_library_file/DIEGO_IDR_CONFIG_MISMATCH_PIPELINE.txt', sep=' ')
list(config['folder_name'])

['D24-10888-6775T_MISMATCH_PIPELINE',
 'D24-10889-6775T_MISMATCH_PIPELINE',
 'D24-10890-6775T_MISMATCH_PIPELINE',
 'D24-10891-6775T_MISMATCH_PIPELINE',
 'D24-10892-6775T_MISMATCH_PIPELINE',
 'D24-10893-6775T_MISMATCH_PIPELINE']

In [15]:
#example_inputs
samp = ['D24-10888-6775T_MISMATCH_PIPELINE',
 'D24-10889-6775T_MISMATCH_PIPELINE',
 'D24-10890-6775T_MISMATCH_PIPELINE',
 'D24-10891-6775T_MISMATCH_PIPELINE',
 'D24-10892-6775T_MISMATCH_PIPELINE',
 'D24-10893-6775T_MISMATCH_PIPELINE']

sample_id = ['sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6']

#filepath for crispresso data
fp = 'example_data/crispresso'

edit_dict = crispresso_compiler(samp, sample_id, fp)

In [16]:
#can then access editing through 
edit_dict['sample1']

,Guide_ID,Reads_in_input,Reads_lowqual,Reads_aligned_all_amplicons,Reads_aligned_WT,Reads_aligned_HDR,Reads_unaligned,WT,correct_edit,target_base_edit,...,byproduct_INDEL,byproduct_sub,byproduct_ambiguous,corr_perc,target_base_edit_perc,WT_perc,byproduct_all_perc,byproduct_INDEL_perc,byproduct_sub_perc,byproduct_ambiguous_perc
0,sg_1,1224,0,1224,33,1189,2,25,99,1189,...,44,1033,23,8.088235,97.140523,2.042484,89.869281,3.594771,84.395425,1.879085
1,sg_10,1694,0,1694,846,847,1,16,0,847,...,32,1623,23,0.000000,50.000000,0.944510,99.055490,1.889020,95.808737,1.357733
2,sg_100,581,0,581,5,564,12,0,1,564,...,4,550,26,0.172117,97.074010,0.000000,99.827883,0.688468,94.664372,4.475043
3,sg_1000,271,8,263,32,226,5,6,0,226,...,57,162,38,0.000000,85.931559,2.281369,97.718631,21.673004,61.596958,14.448669
4,sg_1001,1104,20,1084,23,1039,22,18,72,1039,...,46,881,67,6.642066,95.848708,1.660517,91.697417,4.243542,81.273063,6.180812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667,sg_995,890,46,844,56,772,16,18,94,772,...,48,605,79,11.137441,91.469194,2.132701,86.729858,5.687204,71.682464,9.360190
1668,sg_996,2071,14,2057,66,1970,21,29,0,1970,...,2,1947,79,0.000000,95.770540,1.409820,98.590180,0.097229,94.652406,3.840544
1669,sg_997,1835,1,1834,995,837,2,111,25,837,...,9,1657,32,1.363141,45.637950,6.052345,92.584515,0.490731,90.348964,1.744820
1670,sg_998,4,0,4,0,4,0,0,4,4,...,0,0,0,100.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Merging sensor editing results

It's often useful to merge sensor editing results from the same time-point to make a maximum likelihood estimate of sensor editing:

In [17]:
def MLE_merge(edit_dict, samps_to_merge, new_name):
    """ 
    combine replicates to generate best estimate of sensor editing
    only works for combinations of 3 replicates (could be modified)
    """
    cols_to_add = ['Reads_in_input', 'Reads_lowqual',
       'Reads_aligned_all_amplicons', 'Reads_aligned_WT', 'Reads_aligned_HDR',
       'Reads_unaligned', 'WT', 'correct_edit', 'target_base_edit',
       'byproduct_all', 'byproduct_INDEL', 'byproduct_sub',
       'byproduct_ambiguous',]
    
    comb_holder = []
    for k in samps_to_merge:
        a = edit_dict[k[0]]
        b = edit_dict[k[1]]
        c = edit_dict[k[2]]

        #check that guide indeces match
        assert list(a['Guide_ID'])==list(b['Guide_ID'])==list(c['Guide_ID']), 'indexes dont match'

        a2 = a[cols_to_add]
        b2 = b[cols_to_add]
        c2 = c[cols_to_add]

        comb = a2+b2+c2

        comb['Guide_ID'] = a['Guide_ID']

        #and reorganize columns
        col_order = ['Guide_ID'] + cols_to_add
        out = comb[col_order]

        #and then calcualte percentages
        out['corr_perc'] = 100*(out['correct_edit']/out['Reads_aligned_all_amplicons'])
        out['target_base_edit_perc'] = 100*(out['target_base_edit']/out['Reads_aligned_all_amplicons'])
        out['WT_perc'] = 100*(out['WT']/out['Reads_aligned_all_amplicons'])
        out['byproduct_all_perc'] =  100*(out['byproduct_all']/out['Reads_aligned_all_amplicons'])
        out['byproduct_INDEL_perc'] =  100*(out['byproduct_INDEL']/out['Reads_aligned_all_amplicons'])
        out['byproduct_sub_perc'] =  100*(out['byproduct_sub']/out['Reads_aligned_all_amplicons'])
        out['byproduct_ambiguous_perc'] =100*(out['byproduct_ambiguous']/out['Reads_aligned_all_amplicons'])
        out = out.fillna(0)

        comb_holder.append(out)

    out_dict = dict(zip(new_name, comb_holder))

    return out_dict

In [18]:
#e.g. if sample1, sample2, and sample3 were 
# from the same time-point and you wanted to merge them

samps_to_merge = [['sample1', 'sample2', 'sample3'], ['sample4', 'sample5', 'sample6']]
new_name = ['T0_MLE', 'TF_MLE']

comb_dict = MLE_merge(edit_dict, samps_to_merge, new_name)

/var/folders/5g/xxjcy2g531n02hnyb6d8qdsr0000gn/T/ipykernel_57015/3541396037.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out['corr_perc'] = 100*(out['correct_edit']/out['Reads_aligned_all_amplicons'])
/var/folders/5g/xxjcy2g531n02hnyb6d8qdsr0000gn/T/ipykernel_57015/3541396037.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out['target_base_edit_perc'] = 100*(out['target_base_edit']/out['Reads_aligned_all_amplicons'])
/var/folders/5g/xxjcy2g531n02hnyb6d8qdsr0000gn/T/ipykernel_57015/3541396037.py

In [19]:
#and accessing it
comb_dict['T0_MLE']

,Guide_ID,Reads_in_input,Reads_lowqual,Reads_aligned_all_amplicons,Reads_aligned_WT,Reads_aligned_HDR,Reads_unaligned,WT,correct_edit,target_base_edit,...,byproduct_INDEL,byproduct_sub,byproduct_ambiguous,corr_perc,target_base_edit_perc,WT_perc,byproduct_all_perc,byproduct_INDEL_perc,byproduct_sub_perc,byproduct_ambiguous_perc
0,sg_1,4002,0,4002,95,3902,5,52,286,3902,...,70,3503,91,7.146427,97.501249,1.299350,91.554223,1.749125,87.531234,2.273863
1,sg_10,6209,2,6207,2772,3429,6,68,0,3429,...,73,6007,59,0.000000,55.244079,1.095537,98.904463,1.176092,96.777831,0.950540
2,sg_100,1453,0,1453,18,1399,36,3,2,1399,...,21,1372,55,0.137646,96.283551,0.206469,99.655884,1.445286,94.425327,3.785272
3,sg_1000,1153,37,1116,103,965,48,13,5,965,...,168,643,287,0.448029,86.469534,1.164875,98.387097,15.053763,57.616487,25.716846
4,sg_1001,3291,70,3221,126,3050,45,21,138,3050,...,183,2622,257,4.284384,94.691090,0.651971,95.063645,5.681465,81.403291,7.978889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667,sg_995,2792,106,2686,218,2362,106,131,438,2362,...,90,1726,301,16.306776,87.937453,4.877141,78.816083,3.350707,64.259121,11.206255
1668,sg_996,7681,35,7646,266,7322,58,82,1,7322,...,38,7210,315,0.013079,95.762490,1.072456,98.914465,0.496992,94.297672,4.119801
1669,sg_997,7577,1,7576,3661,3892,23,377,90,3892,...,76,6869,164,1.187962,51.372756,4.976241,93.835797,1.003168,90.667899,2.164731
1670,sg_998,65,0,65,0,65,0,0,5,65,...,0,60,0,7.692308,100.000000,0.000000,92.307692,0.000000,92.307692,0.000000


# MAGeCK example

Example of taking the counts files and making them useable for MAGeCK. We want a table that has columns of samples and rows of guide counts.

In [20]:
config_df = pd.read_csv('example_config_and_library_file/DIEGO_IDR_CONFIG_MISMATCH_PIPELINE.txt', sep=' ')


sample_id = ['TFIN_REP1', 'TFIN_REP2', 'TFIN_REP3',
            'T0_REP1', 'T0_REP2', 'T0_REP3',]

samp = [i for i in config_df['folder_name']]

samp_dict = dict(zip(samp, sample_id))
samp_dict

{'D24-10888-6775T_MISMATCH_PIPELINE': 'TFIN_REP1',
 'D24-10889-6775T_MISMATCH_PIPELINE': 'TFIN_REP2',
 'D24-10890-6775T_MISMATCH_PIPELINE': 'TFIN_REP3',
 'D24-10891-6775T_MISMATCH_PIPELINE': 'T0_REP1',
 'D24-10892-6775T_MISMATCH_PIPELINE': 'T0_REP2',
 'D24-10893-6775T_MISMATCH_PIPELINE': 'T0_REP3'}

In [24]:
#-------UPDATE THESE 2 PARAMTERS----
fp = 'example_data/counts'
chosen_count_column = 'bc_count' #can also choose "matched_guide_count" or "total_guide_count"

df_holder = []
s_name = []
for i, k in enumerate(samp):
    aa = pd.read_csv(f'{fp}/{k}_count_df.csv')
    aa = aa[['Guide_ID', chosen_count_column]]

    s = samp_dict[k]

    aa = aa.rename(columns = {chosen_count_column:f'{s}'})

    if i>0:
        aa = aa.rename(columns = {'Guide_ID':'Guide_ID_x'})
    df_holder.append(aa)

join_out = pd.concat(df_holder, axis=1)
join_out = join_out[['Guide_ID'] + sample_id]
join_out

,Guide_ID,TFIN_REP1,TFIN_REP2,TFIN_REP3,T0_REP1,T0_REP2,T0_REP3
0,sg_1,1945.0,1721.0,2637.0,1820.0,1899.0,2317.0
1,sg_2,1747.0,1957.0,2777.0,4113.0,2055.0,2394.0
2,sg_3,1940.0,2053.0,2624.0,2573.0,1845.0,1881.0
3,sg_4,2791.0,3349.0,4070.0,2772.0,2515.0,3593.0
4,sg_5,1710.0,1874.0,1991.0,2510.0,2033.0,1912.0
...,...,...,...,...,...,...,...
1667,sg_1668,150.0,176.0,180.0,218.0,17.0,43.0
1668,sg_1669,234.0,249.0,441.0,237.0,78.0,808.0
1669,sg_1670,168.0,397.0,202.0,486.0,169.0,727.0
1670,sg_1671,159.0,274.0,214.0,68.0,600.0,249.0


In [28]:
#and then add in the gene information as well
df = pd.read_csv('example_config_and_library_file/IDR_ABE_library_columns_renamed.csv')
guide_gene = dict(zip(df['gRNA_id'], df['Hugo_Symbol']))

genes = []
for i, val in join_out.iterrows():

    k = val['Guide_ID']
    gene = guide_gene[k]

    #make all the neutral guides have the same name
    if 'controlsgRNA' in gene:
        gene = 'controlsgRNA'
    genes.append(gene)

join_out['gene'] = genes
join_out = join_out.rename(columns = {'Guide_ID':'sgRNA'})
join_out = join_out[['sgRNA', 'gene'] + sample_id]
join_out

#the first 2 column names need to be 'sgRNA' and 'gene'

,sgRNA,gene,TFIN_REP1,TFIN_REP2,TFIN_REP3,T0_REP1,T0_REP2,T0_REP3
0,sg_1,ABL1,1945.0,1721.0,2637.0,1820.0,1899.0,2317.0
1,sg_2,ABL1,1747.0,1957.0,2777.0,4113.0,2055.0,2394.0
2,sg_3,ABL1,1940.0,2053.0,2624.0,2573.0,1845.0,1881.0
3,sg_4,ABL2,2791.0,3349.0,4070.0,2772.0,2515.0,3593.0
4,sg_5,AFF1,1710.0,1874.0,1991.0,2510.0,2033.0,1912.0
...,...,...,...,...,...,...,...,...
1667,sg_1668,controlsgRNA,150.0,176.0,180.0,218.0,17.0,43.0
1668,sg_1669,controlsgRNA,234.0,249.0,441.0,237.0,78.0,808.0
1669,sg_1670,controlsgRNA,168.0,397.0,202.0,486.0,169.0,727.0
1670,sg_1671,controlsgRNA,159.0,274.0,214.0,68.0,600.0,249.0


In [29]:
#and finally save it for running on mageck

join_out.to_csv('guide_counts_IDR_ABE.txt', sep='\t', index=False)